In [21]:
import pandas as pd
pd.set_option("display.max_columns", None)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from scipy import stats
from statsmodels.stats import weightstats as stests
import datetime as dt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline
import os
from sqlalchemy import create_engine
import time
from datetime import datetime, date, time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
import scipy.stats as st

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet

In [7]:
df = pd.read_csv('df_modeling_sample.csv')

In [8]:
df

,Unnamed: 0,distance,weather_delay_mean,weather_delay_mean.1,nas_delay_mean,security_delay_mean,late_aircraft_delay_mean,arr_delay,flight_time_afternoon,flight_time_early-morning,flight_time_evening,flight_time_mid-night,flight_time_morning,flight_time_night,flight_season_spring,flight_season_summer,flight_season_winter
0,137835,849.0,0.417663,0.417663,2.399724,0.014523,4.116784,-6.0,0,0,1,0,0,0,0,0,0
1,152581,683.0,0.417663,0.417663,2.399724,0.014523,4.116784,-25.0,0,0,1,0,0,0,1,0,0
2,129181,679.0,0.393928,0.393928,3.393894,0.004749,4.136703,-10.0,0,0,1,0,0,0,0,0,0
3,98113,258.0,0.366193,0.366193,2.031661,0.007455,2.796585,-15.0,0,0,1,0,0,0,0,0,1
4,18577,1205.0,0.708376,0.708376,1.699183,0.035240,4.271195,-14.0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7712,21240,725.0,0.205839,0.205839,1.340548,0.013227,3.848458,98.0,0,0,0,0,0,1,0,0,1
7713,107445,192.0,0.393928,0.393928,3.393894,0.004749,4.136703,13.0,1,0,0,0,0,0,1,0,0
7714,41476,846.0,0.393928,0.393928,3.393894,0.004749,4.136703,-7.0,1,0,0,0,0,0,0,0,1
7715,38304,594.0,0.366193,0.366193,2.031661,0.007455,2.796585,-6.0,0,0,0,0,0,1,0,0,1


In [9]:
df.drop(axis = 1, labels = 'Unnamed: 0', inplace = True)

In [10]:
y = df['arr_delay']

In [11]:
X = df.loc[:, df.columns != 'arr_delay']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.80)

In [18]:
scaler = StandardScaler()

In [19]:
scaled_X_train = scaler.fit_transform(X_train)

In [20]:
scaled_X_test = scaler.transform(X_test)

In [22]:
base_elastic_model = ElasticNet()

In [64]:
param_grid = { 'alpha': [0.0001, 0.001,0.05, 0.1, 1, 5, 10, 100], 'l1_ratio': [0.00000001, 0.0000001, 0.000001, 0.00001,0.0001, 0.001, 0.05,.1,0.2,0.5]}

In [65]:
grid_model = GridSearchCV(base_elastic_model, param_grid=param_grid, scoring = 'neg_mean_squared_error', cv=5, verbose=1)

In [66]:
grid_model.fit(scaled_X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.0001, 0.001, 0.05, 0.1, 1, 5, 10, 100],
                         'l1_ratio': [1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001,
                                      0.05, 0.1, 0.2, 0.5]},
             scoring='neg_mean_squared_error', verbose=1)

In [67]:
grid_model.best_params_

{'alpha': 0.001, 'l1_ratio': 1e-08}

In [68]:
y_pred = grid_model.predict(scaled_X_test)

In [69]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [76]:
mae = mean_absolute_error(y_test, y_pred)

In [77]:
mse = np.sqrt(mean_squared_error(y_test, y_pred))

In [78]:
r2 =metrics.r2_score(y_test, y_pred)

In [83]:
print('Mean Absolute Error: ', mae)
print('Mean Square Error: ', mse)
print('R2 Value: ', r2)

Mean Absolute Error:  18.39822972328786
Mean Square Error:  27.336373783893137
R2 Value:  0.0507999462315073
